In [ ]:
%matplotlib inline
import sys, os, requests
import datetime
import io
import json
import urllib
import zipfile

import pytz

import numpy
import pandas
import requests

from common import calc_mid_weekly_average, smooth_series

In [ ]:
df = pandas.read_excel('owid-covid-data.xlsx', parse_dates=['date'])

In [ ]:
df.columns

In [ ]:
df.groupby('iso_code').mean()[['population']].sort_values('population', ascending=False)

In [ ]:
world = df[df.iso_code == 'OWID_WRL'].set_index('date')[['total_vaccinations']].loc['2020-12-10':, :]
world.columns = ['Total']
world['Daily'], world['Daily7'] = calc_mid_weekly_average(world.Total)
world

In [ ]:
for v in world.Daily7.dropna().values:
    print(int(v))

In [ ]:
us = df[df.iso_code == 'USA'].set_index('date')[['total_vaccinations']].loc['2020-12-12':, :]
us.columns = ['Total']
us['Raw'] = us.Total
us.Total = us.Total.fillna(method='ffill').fillna(0.0)
us.Total = smooth_series(us.Total)
us['Daily'], us['Daily7'] = calc_mid_weekly_average(us.Total)
us.loc['2020-12-12':, :].head(20)

In [ ]:
for v in us.Daily7.dropna().values:
    print(int(v))

In [ ]:
uri = '~/Downloads/us-daily-covid-vaccine-doses-administered.csv'
df2 = pandas.read_csv(uri, parse_dates=['Date'])
df2

In [ ]:
spaz = df2[df2.Entity != 'United States'].groupby('Date').sum().sort_index()
spaz['smooth_vaccinations'] = [int(v) for v in spaz.daily_vaccinations.rolling(window=7, center=True, min_periods=1).mean()]
spaz

In [ ]:
uri = ("https://raw.githubusercontent.com/owid/covid-19-data/master"
       "/public/data/vaccinations/us_state_vaccinations.csv")
df2 = pandas.read_csv(uri, parse_dates=['date'])
df2

In [ ]:
df2.groupby('date').sum()[['total_vaccinations']]